## VISUALIZING YOUR FAVOURITE NBA PLAYER 3 POINTERS GRAPH

Tools we are going to use:

- The NBA API to get the data from any NBA player
- CARTOframes to upload the data seamlessly to CARTO
- The CARTO Python SDK to analyze and create a 3-pointers map
- carto-print to generate a high resolution ready-to-print image


#### Let's start by importing the required modules

In [2]:
import os
import sys
import time

from carto.auth import APIKeyAuthClient
from carto.maps import NamedMapManager
from carto.print import Printer

from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import shotchartdetail

import pandas as pd
from cartoframes.context import CartoContext

#### Time to set the CARTO credentials to use

In [3]:
CARTO_BASE_URL = os.environ['CARTO_API_URL']
CARTO_BASE_URL

'https://aromeu.carto.com/'

In [4]:
CARTO_API_KEY = os.environ['CARTO_API_KEY']
CARTO_USER_NAME = 'aromeu'

#### Set the player name and the teams he has played with

In [54]:
PLAYER_NAME = 'Stephen Curry'
TEAMS_NAME = ['Golden State Warriors']

In [6]:
PLAYER_NAME = 'Russell Westbrook'
TEAMS_NAME = ['Oklahoma City Thunder']

In [25]:
PLAYER_NAME = 'Damian Lillard'
TEAMS_NAME = ['Portland Trail Blazers']

#### Yes, just 10 lines to get all their shoting data

In [55]:
data = []
headers = []
player = players.find_players_by_full_name(PLAYER_NAME)
player_id = player[0]['id']

for team_name in TEAMS_NAME:
  team = teams.find_teams_by_full_name(team_name)
  team_id = team[0]['id']
  shots = shotchartdetail.ShotChartDetail(player_id=player_id, team_id=team_id)
  headers = shots.shot_chart_detail.data['headers']
  data.extend(shots.shot_chart_detail.data['data'])

#### Let's go the data scientist path

In [56]:
df = pd.DataFrame(data, columns=headers)

In [57]:
df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0020900015,17,201939,Stephen Curry,1610612744,Golden State Warriors,1,9,31,...,Left Side Center(LC),16-24 ft.,18,-122,145,1,1,20091028,GSW,HOU
1,Shot Chart Detail,0020900015,413,201939,Stephen Curry,1610612744,Golden State Warriors,4,10,26,...,Left Side Center(LC),16-24 ft.,16,-64,149,1,1,20091028,GSW,HOU
2,Shot Chart Detail,0020900015,453,201939,Stephen Curry,1610612744,Golden State Warriors,4,6,31,...,Right Side Center(RC),16-24 ft.,17,118,123,1,1,20091028,GSW,HOU
3,Shot Chart Detail,0020900015,487,201939,Stephen Curry,1610612744,Golden State Warriors,4,2,25,...,Right Side Center(RC),16-24 ft.,20,121,162,1,1,20091028,GSW,HOU
4,Shot Chart Detail,0020900015,490,201939,Stephen Curry,1610612744,Golden State Warriors,4,1,47,...,Left Side Center(LC),16-24 ft.,18,-125,134,1,1,20091028,GSW,HOU


#### And send the data to your CARTO account

In [58]:
dataset_name = '_'.join(PLAYER_NAME.split(' ')).lower()
cc = CartoContext(base_url=CARTO_BASE_URL, api_key=CARTO_API_KEY)

In [31]:
# 2019 shots
YEAR = '2019'
bool_series = df["GAME_DATE"].str.startswith(YEAR, na = False) 
  
# displaying filtered dataframe 
df = df[bool_series] 

In [59]:
cc.write(df, dataset_name, overwrite=True, lnglat=('LOC_X', 'LOC_Y'))

Params: encode_geom, geom_col and everything in kwargs are deprecated and not being used any more
The following columns were changed in the CARTO copy of this dataframe:
GRID_TYPE -> grid_type
GAME_ID -> game_id
GAME_EVENT_ID -> game_event_id
PLAYER_ID -> player_id
PLAYER_NAME -> player_name
TEAM_ID -> team_id
TEAM_NAME -> team_name
PERIOD -> period
MINUTES_REMAINING -> minutes_remaining
SECONDS_REMAINING -> seconds_remaining
EVENT_TYPE -> event_type
ACTION_TYPE -> action_type
SHOT_TYPE -> shot_type
SHOT_ZONE_BASIC -> shot_zone_basic
SHOT_ZONE_AREA -> shot_zone_area
SHOT_ZONE_RANGE -> shot_zone_range
SHOT_DISTANCE -> shot_distance
LOC_X -> loc_x
LOC_Y -> loc_y
SHOT_ATTEMPTED_FLAG -> shot_attempted_flag
SHOT_MADE_FLAG -> shot_made_flag
GAME_DATE -> game_date
HTM -> htm
VTM -> vtm


/Users/alrocar/Documents/dev/carto-nba/env/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 0cca3cb1-1fda-4f3d-afd5-8b26c3318734
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


Table successfully written to CARTO: https://aromeu.carto.com/dataset/stephen_curry


#### Wait, shots locations are in pixels coordinates. Let's do a hacky trick and let's suppose we are using coordinates in meters

In [60]:
cc.execute("UPDATE {} SET the_geom = st_transform(st_setsrid(st_geometryfromtext('POINT(' || ST_X(the_geom) || ' ' || ST_Y(the_geom) || ')'), 3857), 4326)".format(dataset_name))

/Users/alrocar/Documents/dev/carto-nba/env/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: dd17a2a7-f910-4e91-aebe-dd69c2a46185
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


#### Let's now compose a 7 layers map. If you wonder how I get to imagine this, I used BUILDER + some PostGIS wizardry + a lot (I mean a lot) of trial/error for the styling 

In [61]:
def create_named_map(auth_client, dataset_name, map_name, factor):
    template = {
      "version": "0.0.1",
      "name": map_name,
      "auth": {
        "method": "open"
      },
      "placeholders": {},
      "view": {},
      "layergroup": {
        "version": "1.0.1",
        "layers": [
          {
            "type": "plain",
            "options": {
              "color": "#2d2d2d"
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                              polygon-fill: #2a2a2a;
                              polygon-opacity: 0.9;
                            }
                            #layer::outline {
                              line-width: 2 * %d;
                              line-color: #4edce6;
                              line-opacity: 1;
                            }''' % (factor),
              "sql": '''SELECT 1 AS cartodb_id,
                               the_geom,
                               the_geom_webmercator
                        FROM aromeu.basketball_court
                        UNION
                        SELECT 1 AS cartodb_id,
                               the_geom,
                               the_geom_webmercator
                        FROM aromeu.basketball_court_markers'''
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                              line-width: 30 * %d;
                              line-comp-op: screen;
                              line-opacity: 0.08;
                              [shot_distance >= 31] {
                                line-color: #fff500;
                                line-width: 0.3 * %d;
                              }
                              [shot_distance >= 22][shot_distance < 24] {
                                line-color: #0d3781;
                                line-opacity: 0.1;
                              }

                              [shot_distance >= 24][shot_distance < 26] {
                                line-color: #681a87;
                                line-opacity: 0.1;
                              }

                              [shot_distance >= 26][shot_distance < 28] {
                                line-color: #8a1377;
                              }

                              [shot_distance >= 28][shot_distance < 31] {
                                line-color: #ee29ac;
                              }

                              image-filters: agg-stack-blur(45 * %d, 45 * %d);
                            }''' % (factor, factor, factor, factor),
              "sql": '''WITH a AS (
                          SELECT
                            *,
                            st_transform(the_geom, 3857) as the_geom_webmercator,
                            ST_Length(the_geom::geography) / 1000 AS length
                          FROM (
                            SELECT
                              ST_MakeLine(
                                the_geom,
                                ST_SetSRID(
                                  ST_MakePoint(
                                    -1.53456990177195e-22,
                                    -3.17697838071347e-15
                                  ),
                                  4326
                                )
                              ) AS the_geom,
                              cartodb_id, grid_type, game_id, game_event_id, player_id, player_name, team_id, team_name, period, minutes_remaining, seconds_remaining, event_type, action_type, shot_type, shot_zone_basic, shot_zone_area, shot_zone_range, shot_distance, loc_x, loc_y, shot_attempted_flag, shot_made_flag, game_date, htm, vtm
                            FROM (SELECT * FROM {dataset} WHERE shot_distance >= 22 and shot_distance < 27 and shot_made_flag != 0) _line_analysis
                          ) _cdb_analysis_line_to_single_point
                          ) SELECT * FROM a'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                                line-width: 1 * %d;
                                line-comp-op: screen;
                                line-opacity: 0.7;
                                [shot_distance >= 31] {
                                  line-color: #fff500;
                                }
                                [shot_distance >= 22][shot_distance < 24] {
                                  line-color: #0d3781;
                                }
                                [shot_distance >= 24][shot_distance < 26] {
                                  line-color: #681a87;
                                }

                                [shot_distance >= 26][shot_distance < 28] {
                                  line-color: #8a1377;
                                }

                                [shot_distance >= 28][shot_distance < 31] {
                                  line-color: #ee29ac;
                                }
                              }''' % (factor),
              "sql": '''WITH a AS (
                          SELECT
                            *,
                            st_transform(the_geom, 3857) as the_geom_webmercator,
                            ST_Length(the_geom::geography) / 1000 AS length
                          FROM (
                            SELECT
                              ST_MakeLine(
                                the_geom,
                                ST_SetSRID(
                                  ST_MakePoint(
                                    -1.53456990177195e-22,
                                    -3.17697838071347e-15
                                  ),
                                  4326
                                )
                              ) AS the_geom,
                              cartodb_id, grid_type, game_id, game_event_id, player_id, player_name, team_id, team_name, period, minutes_remaining, seconds_remaining, event_type, action_type, shot_type, shot_zone_basic, shot_zone_area, shot_zone_range, shot_distance, loc_x, loc_y, shot_attempted_flag, shot_made_flag, game_date, htm, vtm
                            FROM (SELECT * FROM {dataset} WHERE shot_distance >= 22 and shot_distance < 27 and shot_made_flag != 0) _line_analysis
                          ) _cdb_analysis_line_to_single_point
                          ) SELECT * FROM a'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                                line-width: 6 * %d;
                                line-comp-op: screen;
                                line-opacity: 0.2;
                                  line-color: #fff500;
                                  image-filters: agg-stack-blur(18 * %d, 18 * %d);

                              }''' % (factor, factor, factor),
              "sql": '''WITH a AS (
                        SELECT
                          *,
                          st_transform(the_geom, 3857) as the_geom_webmercator,
                          ST_Length(the_geom::geography) / 1000 AS length
                        FROM (
                          SELECT
                            ST_MakeLine(
                              the_geom,
                              ST_SetSRID(
                                ST_MakePoint(
                                  -1.53456990177195e-22,
                                  -3.17697838071347e-15
                                ),
                                4326
                              )
                            ) AS the_geom,
                            cartodb_id, grid_type, game_id, game_event_id, player_id, player_name, team_id, team_name, period, minutes_remaining, seconds_remaining, event_type, action_type, shot_type, shot_zone_basic, shot_zone_area, shot_zone_range, shot_distance, loc_x, loc_y, shot_attempted_flag, shot_made_flag, game_date, htm, vtm
                          FROM (SELECT * FROM {dataset} WHERE shot_distance >= 27 and shot_made_flag != 0) _line_analysis
                        ) _cdb_analysis_line_to_single_point
                        ), points AS
                          ( SELECT cartodb_id , loc_x, loc_y, shot_distance,
                                   ST_StartPoint(ST_LineMerge(the_geom_webmercator)) AS p1 ,
                                   ST_EndPoint(ST_LineMerge(the_geom_webmercator)) AS p2
                           FROM a AS q2),
                             mid AS
                          (SELECT *,
                                  ST_SetSRID(ST_MakePoint((ST_X(p2) - ST_X(p1))/2 + ST_X(p1), (ST_Y(p2) - ST_Y(p1))/3 + ST_Y(p1)), 3857) AS midpoint,
                                  PI()/2 - ST_Azimuth(p1, p2) AS angle,
                                  ST_Distance(p1, p2)/6 AS radius
                           FROM points),
                             third AS
                          (SELECT *,
                                  ST_Translate(midpoint, sign(loc_x) *0.15 *sin(angle)*radius, 0 *cos(angle)*radius) AS p3
                           FROM mid)
                        SELECT *,
                               ST_SetSRID(ST_CurveToLine('CIRCULARSTRING( ' || ST_X(p1) || ' ' || ST_Y(p1) || ',' || ST_X(p3) || ' ' || ST_Y(p3) || ',' || ST_X(p2) || ' ' || ST_Y(p2) || ')'), 3857) AS the_geom_webmercator
                        FROM third'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                              line-width: 1.5 * %d;
                              line-comp-op: lighten;
                              line-opacity: 0.7;
                                  line-color: #fff500;

                            }''' % (factor),
              "sql": '''WITH a AS (
                          SELECT
                            *,
                            st_transform(the_geom, 3857) as the_geom_webmercator,
                            ST_Length(the_geom::geography) / 1000 AS length
                          FROM (
                            SELECT
                              ST_MakeLine(
                                the_geom,
                                ST_SetSRID(
                                  ST_MakePoint(
                                    -1.53456990177195e-22,
                                    -3.17697838071347e-15
                                  ),
                                  4326
                                )
                              ) AS the_geom,
                              cartodb_id, grid_type, game_id, game_event_id, player_id, player_name, team_id, team_name, period, minutes_remaining, seconds_remaining, event_type, action_type, shot_type, shot_zone_basic, shot_zone_area, shot_zone_range, shot_distance, loc_x, loc_y, shot_attempted_flag, shot_made_flag, game_date, htm, vtm
                            FROM (SELECT * FROM {dataset} WHERE shot_distance >= 27 and shot_made_flag != 0) _line_analysis
                          ) _cdb_analysis_line_to_single_point
                          ), points AS
                            ( SELECT cartodb_id , loc_x, loc_y, shot_distance,
                                     ST_StartPoint(ST_LineMerge(the_geom_webmercator)) AS p1 ,
                                     ST_EndPoint(ST_LineMerge(the_geom_webmercator)) AS p2
                             FROM a AS q2),
                               mid AS
                            (SELECT *,
                                    ST_SetSRID(ST_MakePoint((ST_X(p2) - ST_X(p1))/2 + ST_X(p1), (ST_Y(p2) - ST_Y(p1))/3 + ST_Y(p1)), 3857) AS midpoint,
                                    PI()/2 - ST_Azimuth(p1, p2) AS angle,
                                    ST_Distance(p1, p2)/6 AS radius
                             FROM points),
                               third AS
                            (SELECT *,
                                    ST_Translate(midpoint, sign(loc_x) *0.15 *sin(angle)*radius, 0 *cos(angle)*radius) AS p3
                             FROM mid)
                          SELECT *,
                                 ST_SetSRID(ST_CurveToLine('CIRCULARSTRING( ' || ST_X(p1) || ' ' || ST_Y(p1) || ',' || ST_X(p3) || ' ' || ST_Y(p3) || ',' || ST_X(p2) || ' ' || ST_Y(p2) || ')'), 3857) AS the_geom_webmercator
                          FROM third'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer['mapnik::geometry_type'=1] {
                                marker-fill: #fff;
                                marker-width: 12 * %d;
                                marker-line-color: #fff;
                                marker-line-width: 0;
                                marker-line-opacity: 1;
                                marker-opacity: 0.6;
                                marker-type: ellipse;
                                marker-placement: point;
                                marker-allow-overlap: true;
                                marker-comp-op: lighten;
                                marker-clip: false;
                                marker-multi-policy: largest;
                                image-filters: agg-stack-blur(18 * %d, 18 * %d);
                              }''' % (factor, factor, factor),
              "sql": '''with a as (select action_type,game_event_id,game_id,minutes_remaining,period,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date, the_geom_webmercator, the_geom from {dataset})
SELECT 1 as cartodb_id, * FROM a WHERE (shot_distance >= 22 and (shot_zone_area like '%(R)' or shot_zone_area like '%(L)') and shot_zone_basic != 'Mid-Range') or (shot_distance >= 24 and shot_zone_basic != 'Mid-Range') and shot_type = '3PT Field Goal' and shot_made_flag != 0'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer['mapnik::geometry_type'=1] {
                                marker-fill: #fff;
                                marker-width: 9 * %d;
                                marker-line-color: #fff;
                                marker-line-width: 3 * %d;
                                marker-line-opacity: 1;
                                marker-opacity: 0.3;
                                marker-type: ellipse;
                                marker-placement: point;
                                marker-allow-overlap: true;
                                marker-comp-op: lighten;
                                marker-clip: false;
                                marker-multi-policy: largest;
                              }''' % (factor, factor),
              "sql": '''with a as (select action_type,game_event_id,game_id,minutes_remaining,period,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date, the_geom_webmercator, the_geom from {dataset})
SELECT 1 as cartodb_id, * FROM a WHERE (shot_distance >= 22 and (shot_zone_area like '%(R)' or shot_zone_area like '%(L)') and shot_zone_basic != 'Mid-Range') or (shot_distance >= 24 and shot_zone_basic != 'Mid-Range') and shot_type = '3PT Field Goal' and shot_made_flag != 0'''.format(dataset=dataset_name)
            }
          }
        ]
      }
    }

    named_map_manager = NamedMapManager(auth_client)

    try:
      named_map = named_map_manager.get(map_name)
      if named_map is not None:
          named_map.client = auth_client
          named_map.delete()
    except Exception as e:
      #ignore
      print(e)

    return named_map_manager.create(template=template)

#### This is how we authenticate the CARTO Python SDK

In [62]:
auth_client = APIKeyAuthClient(CARTO_BASE_URL, CARTO_API_KEY)

#### This is one of the things I love the most from CARTO: prototype with BUILDER + then template your map and finally use APIs to produce maps programmatically

For this specific case, we have parameterized the line and markers widths, so with a single template, we can produce maps that we can use to share a screenshot (with 72DPI) or to export for high resolution printing (with 300DPI).

Let's go for the poster printing

In [63]:
DPI = 300
FACTOR = DPI / 72.0
map_name = 'tpl_' + dataset_name + str(int(round(time.time() * 1000)))
create_named_map(auth_client, dataset_name, map_name, FACTOR)

Resource not found: https://aromeu.carto.com/api/v1/map/named/tpl_stephen_curry1557482207033?api_key=424dec8b179567aace6ef7b229c9afa1d78d68e7&client=cps-1.5.dev0


#### Aaaand we're mostly done. Let's export a huge-high-resolution image

In [64]:
map = {
  'username': CARTO_USER_NAME,
  'map_id': map_name,
  'width': 120,
  'height': 80,
  'dpi': DPI,
  'zoom': 18,
  'bounds': {"ne":[-0.000977916642979147,-0.004578593652695418],"sw":[0.004981951781790824,0.004288789350539447]},
  'api_key': CARTO_API_KEY
}

p = Printer(map['username'], map['map_id'], map['api_key'], map['width'], map['height'], map['zoom'], map['bounds'], map['dpi'], 'RGBA')
image_path = p.export('.')

In [65]:
image_path

'./aromeu_tpl_stephen_curry1557482207033_20190510115648.png'

#### How it looks like??

![](aromeu_tpl_stephen_curry1557482207033_20190510115648.png)

#### Clean some stuff and close the door when you leave, please

In [15]:
named_map_manager = NamedMapManager(auth_client)

try:
  named_map = named_map_manager.get(map_name)
  if named_map is not None:
      named_map.client = auth_client
      named_map.delete()
except Exception as e:
  #ignore
  print(e)